In [1]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import requests
import time
import ssl
from geopy.geocoders import Nominatim
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [2]:
munich_lat_long = pd.read_excel('https://github.com/katarzyna-pi/Coursera_Capstone/blob/master/Munich_latlong.xlsx?raw=true')
munich_lat_long.head()

,PostalCode,Latitude,Longitude
0,80331,48.135964,11.572905
1,80333,48.145190,11.568744
2,80335,48.145558,11.553952
3,80336,48.132633,11.554171
4,80337,48.126389,11.558385


In [3]:
munich_plz = pd.read_excel('https://github.com/katarzyna-pi/Coursera_Capstone/blob/master/Munich_plz.xlsx?raw=true')
munich_plz.head()

,PostalCode,District,Neighborhood
0,80997,Allach-Untermenzing,Allach-Untermenzing
1,80999,Allach-Untermenzing,Allach-Untermenzing
2,81247,Allach-Untermenzing,Allach-Untermenzing
3,80331,Altstadt-Lehel,Altstadt-Lehel
4,80333,Altstadt-Lehel,Altstadt-Lehel


In [4]:
munich_df = pd.merge(munich_lat_long, munich_plz, on='PostalCode')
munich_df.head()

,PostalCode,Latitude,Longitude,District,Neighborhood
0,80331,48.135964,11.572905,Altstadt-Lehel,Altstadt-Lehel
1,80333,48.145190,11.568744,Altstadt-Lehel,Altstadt-Lehel
2,80335,48.145558,11.553952,Schwanthalerhöhe,Schwanthalerhöhe
3,80336,48.132633,11.554171,Ludwigsvorstadt-Isarvorstadt,Ludwigsvorstadt-Isarvorstadt
4,80337,48.126389,11.558385,Ludwigsvorstadt-Isarvorstadt,Ludwigsvorstadt-Isarvorstadt


In [5]:
# Using geopy to get the latitude and longitude values of Toronto
address = 'Munich,Germany'
geolocator = Nominatim(user_agent="munich_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Coordinates of Munich are {}, {}. '.format(latitude, longitude))

Coordinates of Munich are 48.1371079, 11.5753822. 


In [6]:
# creating a map of Munich using latitude and longitude values
munich_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# adding markers
for lat, lng, district, in zip(munich_df['Latitude'], munich_df['Longitude'], munich_df['District']):
    label = '{},'.format(district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(munich_map)
    
munich_map

In [7]:
CLIENT_ID = 'UM2PPVQ0FZT5A3W12BQNEUUDE4XARLR3R2EUYT1YGJFWPQPT'
CLIENT_SECRET = 'TDUFRWNPUNWOI24T1WE4DQJDDMD4XGAZBIHAUQLXTYE3OPSL' 
VERSION = '20180604'
LIMIT = 30

In [8]:
# first District in munich_df
munich_df.loc[0, 'District']

'Altstadt-Lehel'

In [9]:
# getting Altstadt-Lehel's latitude and longitude values
Altstadt_Lehel_latitude = munich_df.loc[0, 'Latitude']
Altstadt_Lehel_longitude = munich_df.loc[0, 'Longitude']
print('Altstadt-Lehel\'s latitude and longitude values are {}, {}.'.format(Altstadt_Lehel_latitude, Altstadt_Lehel_longitude))

Altstadt-Lehel's latitude and longitude values are 48.1359642378, 11.5729048503.


In [10]:
# getting the top 100 venues that are in Altstadt-Lehel within a radius of 700 meteres
no_of_venues = 100
radius = 700
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, Altstadt_Lehel_latitude, Altstadt_Lehel_longitude, radius, no_of_venues)

response = requests.get(url).json()
response

{'meta': {'code': 200, 'requestId': '5ec503fd9c6f59001bcdcad3'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Hackenviertel',
  'headerFullLocation': 'Hackenviertel, Munich',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 235,
  'suggestedBounds': {'ne': {'lat': 48.14226424410001,
    'lng': 11.582327342277884},
   'sw': {'lat': 48.129664231499994, 'lng': 11.563482358322117}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ade0ccef964a520246921e3',
       'name': 'Marienplatz',
       'location': {'address': 'Marienplatz',
        'lat': 48.1371253550687,
        'lng': 11.5754829819069,
        'distance': 231,
        'postalCode': '80331',
        'cc': 'D

In [11]:
# function to extract the venue's category
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
#cleaning the response json and structuring it into pandas dataframe
venues = response['response']['groups'][0]['items']

# flatten JSON
nearby_venues = json_normalize(venues)

# filtering only needed columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Marienplatz,Plaza,48.137125,11.575483
1,Asamkirche (St. Johann Nepomuk),Church,48.135053,11.569746
2,Galeria Gourmet,Gourmet Shop,48.137432,11.573217
3,Viktualienmarkt,Farmers Market,48.135296,11.576368
4,Kustermann,Department Store,48.136242,11.574897


In [15]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=700):
    
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                            'District Latitude',
                            'District Longitude',
                            'Venue',
                            'Venue Latitude',
                            'Venue Longitude',
                            'Venue Category']
    
    return(nearby_venues)

In [17]:
munich_venues = getNearbyVenues(names=munich_df['District'],
                                 latitudes=munich_df['Latitude'],
                                 longitudes=munich_df['Longitude'])

Altstadt-Lehel
Altstadt-Lehel
Schwanthalerhöhe
Ludwigsvorstadt-Isarvorstadt
Ludwigsvorstadt-Isarvorstadt
Schwanthalerhöhe
Altstadt-Lehel
Altstadt-Lehel
Maxvorstadt
Neuhausen-Nymphenburg
Neuhausen-Nymphenburg
Moosach
Moosach
Neuhausen-Nymphenburg
Laim
Laim
Hadern
Schwabing-West
Schwabing-West
Maxvorstadt
Maxvorstadt
Schwabing-West
Maxvorstadt
Schwabing-West
Schwabing-Freimann
Schwabing-Freimann
Schwabing-Freimann
Milbertshofen-Am Hart
Feldmoching-Hasenbergl
Feldmoching-Hasenbergl
Milbertshofen-Am Hart
Schwabing-Freimann
Moosach
Moosach
Feldmoching-Hasenbergl
Allach-Untermenzing
Allach-Untermenzing
Pasing-Obermenzing
Aubing-Lochhausen-Langwied
Aubing-Lochhausen-Langwied
Allach-Untermenzing
Aubing-Lochhausen-Langwied
Sendling
Sendling
Sendling
Hadern
Hadern
Sendling
Thalkirchen
Thalkirchen
Thalkirchen
Thalkirchen
Obergiesing
Au-Haidhausen
Untergiesing-Harlaching
Untergiesing-Harlaching
Obergiesing
Obergiesing
Au-Haidhausen
Au-Haidhausen
Ramersdorf-Perlach
Berg am Laim
Bogenhausen
Au-Haidh

In [18]:
print(munich_venues.shape)
munich_venues.head()

(1465, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Altstadt-Lehel,48.135964,11.572905,Marienplatz,48.137125,11.575483,Plaza
1,Altstadt-Lehel,48.135964,11.572905,Asamkirche (St. Johann Nepomuk),48.135053,11.569746,Church
2,Altstadt-Lehel,48.135964,11.572905,Galeria Gourmet,48.137432,11.573217,Gourmet Shop
3,Altstadt-Lehel,48.135964,11.572905,Viktualienmarkt,48.135296,11.576368,Farmers Market
4,Altstadt-Lehel,48.135964,11.572905,Kustermann,48.136242,11.574897,Department Store


In [19]:
# number of venues for each district
munich_venues.groupby('District')['Venue'].count()

District
Allach-Untermenzing              28
Altstadt-Lehel                  120
Au-Haidhausen                   104
Aubing-Lochhausen-Langwied       25
Berg am Laim                     63
Bogenhausen                      81
Feldmoching-Hasenbergl           17
Hadern                           35
Laim                             40
Ludwigsvorstadt-Isarvorstadt     60
Maxvorstadt                     120
Milbertshofen-Am Hart            34
Moosach                          63
Neuhausen-Nymphenburg            78
Obergiesing                      53
Pasing-Obermenzing               20
Ramersdorf-Perlach               32
Schwabing-Freimann               95
Schwabing-West                  120
Schwanthalerhöhe                 60
Sendling                        100
Thalkirchen                      66
Trudering-Riem                   34
Untergiesing-Harlaching          17
Name: Venue, dtype: int64

In [20]:
# number of unique categories of venues
print('There are {} unique categories.'.format(len(munich_venues['Venue Category'].unique())))

There are 235 unique categories.


In [21]:
# one hot encoding
munich_onehot = pd.get_dummies(munich_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
munich_onehot['District'] = munich_venues['District'] 

# move neighborhood column to the first column
fixed_columns = [munich_onehot.columns[-1]] + list(munich_onehot.columns[:-1])
munich_onehot = munich_onehot[fixed_columns]

munich_onehot.head()

,District,Afghan Restaurant,African Restaurant,American Restaurant,Aquarium,Arcade,Art Museum,Asian Restaurant,Athletics & Sports,Austrian Restaurant,...,Trattoria/Osteria,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfall,Wine Bar,Wine Shop,Xinjiang Restaurant,Yoga Studio
0,Altstadt-Lehel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Altstadt-Lehel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Altstadt-Lehel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Altstadt-Lehel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Altstadt-Lehel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
munich_onehot.shape

(1465, 236)

In [23]:
munich_grouped = munich_onehot.groupby('District').sum().reset_index()
munich_grouped

,District,Afghan Restaurant,African Restaurant,American Restaurant,Aquarium,Arcade,Art Museum,Asian Restaurant,Athletics & Sports,Austrian Restaurant,...,Trattoria/Osteria,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfall,Wine Bar,Wine Shop,Xinjiang Restaurant,Yoga Studio
0,Allach-Untermenzing,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,Altstadt-Lehel,2,0,0,0,0,6,2,0,0,...,1,0,0,1,1,1,0,0,0,0
2,Au-Haidhausen,0,0,0,0,0,0,1,1,0,...,1,0,3,1,2,0,1,0,0,0
3,Aubing-Lochhausen-Langwied,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Berg am Laim,0,0,0,0,0,0,2,1,0,...,1,0,2,0,0,0,0,0,0,0
5,Bogenhausen,0,0,0,0,0,1,1,0,0,...,0,2,0,0,2,0,0,0,0,0
6,Feldmoching-Hasenbergl,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7,Hadern,0,0,0,0,0,0,1,0,0,...,2,0,0,0,0,0,0,0,0,0
8,Laim,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Ludwigsvorstadt-Isarvorstadt,0,1,0,0,0,0,1,0,1,...,0,0,0,0,1,0,0,1,2,0


In [24]:
munich_grouped.shape

(24, 236)

In [63]:
munich_venues_sum = munich_grouped.sum(axis = 0)
munich_venues_sum_df = munich_venues_sum.to_frame()
munich_venues_sum_df2 = munich_venues_sum_df.drop(['District'])
pd.set_option('display.max_rows', 235)
print(munich_venues_sum_df2)

                                0
Afghan Restaurant               8
African Restaurant              1
American Restaurant             1
Aquarium                        1
Arcade                          1
Art Museum                      9
Asian Restaurant               17
Athletics & Sports              7
Austrian Restaurant             7
Auto Dealership                 1
Automotive Shop                 3
BBQ Joint                       5
Bagel Shop                      2
Bakery                         61
Bank                           11
Bar                            23
Basketball Stadium              1
Bavarian Restaurant             8
Beach                           3
Bed & Breakfast                 1
Beer Bar                        2
Beer Garden                    16
Beer Store                      5
Big Box Store                   2
Bistro                          8
Boarding House                  1
Bookstore                       4
Bosnian Restaurant              1
Botanical Gard

In [25]:
num_top_venues = 5

for hood in munich_grouped['District']:
    print("[--------"+hood+"--------]")
    temp = munich_grouped[munich_grouped['District'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print("\n")

[--------Allach-Untermenzing--------]
         venue  freq
0        Hotel   4.0
1       Bakery   4.0
2         Park   3.0
3     Bus Stop   2.0
4  Supermarket   2.0


[--------Altstadt-Lehel--------]
                venue  freq
0                Café  14.0
1  Italian Restaurant   7.0
2               Plaza   7.0
3          Art Museum   6.0
4   German Restaurant   5.0


[--------Au-Haidhausen--------]
                venue  freq
0   German Restaurant   6.0
1  Italian Restaurant   5.0
2    Greek Restaurant   4.0
3                Café   4.0
4          Restaurant   4.0


[--------Aubing-Lochhausen-Langwied--------]
              venue  freq
0          Bus Stop   4.0
1       Supermarket   3.0
2              Park   2.0
3  Asian Restaurant   2.0
4            Bakery   2.0


[--------Berg am Laim--------]
                venue  freq
0         Supermarket   9.0
1           Drugstore   6.0
2              Bakery   4.0
3  Italian Restaurant   3.0
4            Bus Stop   3.0


[--------Bogenhausen-----

In [26]:
#function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
# create a dataframe to display top 10 venues for each neighborhood
num_top_venues = 10
indicators = ['st', 'nd', 'rd', 'th']

columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
         columns.append('{}th Most Common Venue'.format(ind+1))
        

# create the new dataframe
neighborhood_venues_sorted = pd.DataFrame(columns=columns)
neighborhood_venues_sorted['District'] = munich_grouped['District']

for ind in np.arange(munich_grouped.shape[0]):
    neighborhood_venues_sorted.iloc[ind, 1:] = return_most_common_venues(munich_grouped.iloc[ind, :], num_top_venues)

print(neighborhood_venues_sorted.shape)    
neighborhood_venues_sorted

(24, 11)


,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allach-Untermenzing,Hotel,Bakery,Park,German Restaurant,Bus Stop,Supermarket,Tennis Court,Indian Restaurant,Italian Restaurant,Drugstore
1,Altstadt-Lehel,Café,Italian Restaurant,Plaza,Art Museum,German Restaurant,Ice Cream Shop,Pizza Place,History Museum,Restaurant,Cocktail Bar
2,Au-Haidhausen,German Restaurant,Italian Restaurant,Greek Restaurant,Supermarket,Restaurant,Café,Plaza,Bar,Turkish Restaurant,Bakery
3,Aubing-Lochhausen-Langwied,Bus Stop,Supermarket,Bakery,Asian Restaurant,Automotive Shop,Park,Bowling Alley,Light Rail Station,Bed & Breakfast,Big Box Store
4,Berg am Laim,Supermarket,Drugstore,Bakery,Italian Restaurant,Bus Stop,Asian Restaurant,Metro Station,Bavarian Restaurant,Hotel,Turkish Restaurant
5,Bogenhausen,Italian Restaurant,Park,Bakery,Drugstore,Bus Stop,Restaurant,Plaza,Supermarket,Tram Station,Indian Restaurant
6,Feldmoching-Hasenbergl,Supermarket,Plaza,Bus Stop,Greek Restaurant,Café,Lake,Bakery,Beer Garden,Gas Station,Trattoria/Osteria
7,Hadern,Bus Stop,Supermarket,Plaza,Trattoria/Osteria,German Restaurant,Greek Restaurant,Italian Restaurant,Bakery,Metro Station,Food & Drink Shop
8,Laim,Supermarket,Bakery,Tram Station,Greek Restaurant,Restaurant,Drugstore,Doner Restaurant,Bank,Hotel,Brewery
9,Ludwigsvorstadt-Isarvorstadt,Café,Italian Restaurant,German Restaurant,Restaurant,Hotel,Jazz Club,Bavarian Restaurant,Falafel Restaurant,Ice Cream Shop,Xinjiang Restaurant


In [28]:
# set number of clusters
kclusters = 5

munich_grouped_clustering = munich_grouped.drop('District', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(munich_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 4, 2, 1, 0, 0, 1, 1, 1, 1], dtype=int32)

In [29]:
# add clustering labels

neighborhood_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

munich_merged = munich_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
munich_merged = munich_merged.join(neighborhood_venues_sorted.set_index('District'), on='District')

munich_merged.head() # check the last columns!

,PostalCode,Latitude,Longitude,District,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,80331,48.135964,11.572905,Altstadt-Lehel,Altstadt-Lehel,4,Café,Italian Restaurant,Plaza,Art Museum,German Restaurant,Ice Cream Shop,Pizza Place,History Museum,Restaurant,Cocktail Bar
1,80333,48.145190,11.568744,Altstadt-Lehel,Altstadt-Lehel,4,Café,Italian Restaurant,Plaza,Art Museum,German Restaurant,Ice Cream Shop,Pizza Place,History Museum,Restaurant,Cocktail Bar
2,80335,48.145558,11.553952,Schwanthalerhöhe,Schwanthalerhöhe,2,Café,Hotel,Ice Cream Shop,Italian Restaurant,Middle Eastern Restaurant,Sushi Restaurant,Burger Joint,Vietnamese Restaurant,Restaurant,Thai Restaurant
3,80336,48.132633,11.554171,Ludwigsvorstadt-Isarvorstadt,Ludwigsvorstadt-Isarvorstadt,1,Café,Italian Restaurant,German Restaurant,Restaurant,Hotel,Jazz Club,Bavarian Restaurant,Falafel Restaurant,Ice Cream Shop,Xinjiang Restaurant
4,80337,48.126389,11.558385,Ludwigsvorstadt-Isarvorstadt,Ludwigsvorstadt-Isarvorstadt,1,Café,Italian Restaurant,German Restaurant,Restaurant,Hotel,Jazz Club,Bavarian Restaurant,Falafel Restaurant,Ice Cream Shop,Xinjiang Restaurant


In [33]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(munich_merged['Latitude'], munich_merged['Longitude'], munich_merged['District'], munich_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=1.0).add_to(map_clusters)
       
map_clusters

In [34]:
cluster0_raw = munich_merged.loc[munich_merged['Cluster Labels'] == 0, munich_merged.columns[[3] + list(range(5, munich_merged.shape[1]))]]
cluster0 = cluster0_raw.drop_duplicates()
cluster0

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Schwabing-Freimann,0,Supermarket,Hotel,Bus Stop,Italian Restaurant,Bakery,Plaza,Café,Gastropub,Clothing Store,Drugstore
42,Sendling,0,Supermarket,Italian Restaurant,Hotel,Bakery,Drugstore,Bus Stop,German Restaurant,Gastropub,Greek Restaurant,Indian Restaurant
61,Berg am Laim,0,Supermarket,Drugstore,Bakery,Italian Restaurant,Bus Stop,Asian Restaurant,Metro Station,Bavarian Restaurant,Hotel,Turkish Restaurant
62,Bogenhausen,0,Italian Restaurant,Park,Bakery,Drugstore,Bus Stop,Restaurant,Plaza,Supermarket,Tram Station,Indian Restaurant


In [35]:
cluster1_raw = munich_merged.loc[munich_merged['Cluster Labels'] == 1, munich_merged.columns[[3] + list(range(5, munich_merged.shape[1]))]]
cluster1 = cluster1_raw.drop_duplicates()
cluster1

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Ludwigsvorstadt-Isarvorstadt,1,Café,Italian Restaurant,German Restaurant,Restaurant,Hotel,Jazz Club,Bavarian Restaurant,Falafel Restaurant,Ice Cream Shop,Xinjiang Restaurant
11,Moosach,1,Café,Supermarket,Greek Restaurant,Bakery,Bus Stop,Trattoria/Osteria,Tram Station,Light Rail Station,Pool,Gastropub
14,Laim,1,Supermarket,Bakery,Tram Station,Greek Restaurant,Restaurant,Drugstore,Doner Restaurant,Bank,Hotel,Brewery
16,Hadern,1,Bus Stop,Supermarket,Plaza,Trattoria/Osteria,German Restaurant,Greek Restaurant,Italian Restaurant,Bakery,Metro Station,Food & Drink Shop
27,Milbertshofen-Am Hart,1,Museum,Gift Shop,Bus Stop,Pizza Place,Concert Hall,Soccer Field,Shipping Store,Scenic Lookout,Cafeteria,Café
28,Feldmoching-Hasenbergl,1,Supermarket,Plaza,Bus Stop,Greek Restaurant,Café,Lake,Bakery,Beer Garden,Gas Station,Trattoria/Osteria
35,Allach-Untermenzing,1,Hotel,Bakery,Park,German Restaurant,Bus Stop,Supermarket,Tennis Court,Indian Restaurant,Italian Restaurant,Drugstore
37,Pasing-Obermenzing,1,German Restaurant,Hotel,Automotive Shop,Organic Grocery,Pool,Drugstore,Bus Stop,Supermarket,Italian Restaurant,Taverna
38,Aubing-Lochhausen-Langwied,1,Bus Stop,Supermarket,Bakery,Asian Restaurant,Automotive Shop,Park,Bowling Alley,Light Rail Station,Bed & Breakfast,Big Box Store
48,Thalkirchen,1,Bus Stop,Bakery,Drugstore,Fast Food Restaurant,Bank,Supermarket,German Restaurant,BBQ Joint,Italian Restaurant,Ice Cream Shop


In [36]:
cluster2_raw = munich_merged.loc[munich_merged['Cluster Labels'] == 2, munich_merged.columns[[3] + list(range(5, munich_merged.shape[1]))]]
cluster2 = cluster2_raw.drop_duplicates()
cluster2

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Schwanthalerhöhe,2,Café,Hotel,Ice Cream Shop,Italian Restaurant,Middle Eastern Restaurant,Sushi Restaurant,Burger Joint,Vietnamese Restaurant,Restaurant,Thai Restaurant
9,Neuhausen-Nymphenburg,2,Café,German Restaurant,Italian Restaurant,Plaza,Bakery,Vietnamese Restaurant,Pizza Place,Greek Restaurant,Drugstore,Indie Movie Theater
17,Schwabing-West,2,Italian Restaurant,Café,Vietnamese Restaurant,Thai Restaurant,German Restaurant,Coffee Shop,Supermarket,Ice Cream Shop,Greek Restaurant,Trattoria/Osteria
53,Au-Haidhausen,2,German Restaurant,Italian Restaurant,Greek Restaurant,Supermarket,Restaurant,Café,Plaza,Bar,Turkish Restaurant,Bakery


In [37]:
cluster3_raw = munich_merged.loc[munich_merged['Cluster Labels'] == 3, munich_merged.columns[[3] + list(range(5, munich_merged.shape[1]))]]
cluster3 = cluster3_raw.drop_duplicates()
cluster3

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Maxvorstadt,3,Café,Bar,Italian Restaurant,Plaza,Vietnamese Restaurant,German Restaurant,Bakery,Trattoria/Osteria,Ice Cream Shop,Pastry Shop


In [38]:
cluster4_raw = munich_merged.loc[munich_merged['Cluster Labels'] == 4, munich_merged.columns[[3] + list(range(5, munich_merged.shape[1]))]]
cluster4 = cluster4_raw.drop_duplicates()
cluster4

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Altstadt-Lehel,4,Café,Italian Restaurant,Plaza,Art Museum,German Restaurant,Ice Cream Shop,Pizza Place,History Museum,Restaurant,Cocktail Bar


In [74]:
munich_vegetarian = munich_grouped[['District','Vegetarian / Vegan Restaurant']]

In [84]:
munich_vegetarian

,District,Vegetarian / Vegan Restaurant
0,Allach-Untermenzing,0
1,Altstadt-Lehel,1
2,Au-Haidhausen,1
3,Aubing-Lochhausen-Langwied,0
4,Berg am Laim,0
5,Bogenhausen,0
6,Feldmoching-Hasenbergl,0
7,Hadern,0
8,Laim,0
9,Ludwigsvorstadt-Isarvorstadt,0
